In [1]:
import os
import timm
import torch

import numpy as np
import pandas as pd

from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score, top_k_accuracy_score
# from fgvc.utils.utils import set_random_seed

SEED = 777
# set_random_seed(SEED)
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f'Device: {device}')

Device: mps


In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
## Using metadata
train_df = pd.read_csv("./FungiTastic-Mini-Train.csv")
val_df = pd.read_csv("./FungiTastic-Mini-ClosedSet-Val.csv")

# train_df = pd.read_csv("./FungiTastic-Train.csv")
# test_df = pd.read_csv("./FungiTastic-ClosedSet-Val.csv")

labels = sorted(train_df["category_id"].unique())

train_df

,eventDate,year,month,day,habitat,countryCode,scientificName,kingdom,phylum,class,...,observationID,region,district,filename,category_id,metaSubstrate,poisonous,elevation,landcover,biogeographicalRegion
0,2021-02-01,2021,2.0,1.0,Mixed woodland (with coniferous and deciduous ...,DK,Mycena tintinnabulum (Paulet) Quél.,Fungi,Basidiomycota,Agaricomycetes,...,3032614317,Sjælland,Næstved,0-3032614317.JPG,119,wood,0,35.0,5.0,continental
1,2021-02-01,2021,2.0,1.0,Mixed woodland (with coniferous and deciduous ...,DK,Mycena tintinnabulum (Paulet) Quél.,Fungi,Basidiomycota,Agaricomycetes,...,3032614317,Sjælland,Næstved,1-3032614317.JPG,119,wood,0,35.0,5.0,continental
2,2008-09-01,2008,9.0,1.0,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,...,3036761318,Midtjylland,Århus,0-3036761318.JPG,144,jord,0,6.0,10.0,continental
3,2008-09-01,2008,9.0,1.0,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,...,3036761318,Midtjylland,Århus,1-3036761318.JPG,144,jord,0,6.0,10.0,continental
4,2008-09-01,2008,9.0,1.0,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,...,3036761318,Midtjylland,Århus,2-3036761318.JPG,144,jord,0,6.0,10.0,continental
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46837,2021-09-29,2021,9.0,29.0,Unmanaged deciduous woodland,RU,Mycena rosea Gramberg,Fungi,Basidiomycota,Agaricomycetes,...,3429074356,Kaluga,Maloyaroslavetskiy rayon,1-3429074356.JPG,109,jord,1,NaN,NaN,boreal
46838,2021-09-29,2021,9.0,29.0,Unmanaged deciduous woodland,RU,Mycena rosea Gramberg,Fungi,Basidiomycota,Agaricomycetes,...,3429074356,Kaluga,Maloyaroslavetskiy rayon,2-3429074356.JPG,109,jord,1,NaN,NaN,boreal
46839,2021-09-27,2021,9.0,27.0,Deciduous woodland,DK,Russula ochroleuca Fr.,Fungi,Basidiomycota,Agaricomycetes,...,4100099773,Hovedstaden,Halsnæs,0-4100099773.JPG,179,jord,0,0.0,17.0,continental
46840,2021-09-27,2021,9.0,27.0,Deciduous woodland,DK,Russula ochroleuca Fr.,Fungi,Basidiomycota,Agaricomycetes,...,4100099773,Hovedstaden,Halsnæs,1-4100099773.JPG,179,jord,0,0.0,17.0,continental


In [4]:
TRAIN_IMAGE_DIR = "/Users/lukaspicek/Downloads/images/FungiTastic-Mini/train/500p"
train_df["image_path"] = train_df.filename.apply(
    lambda filename: os.path.join(TRAIN_IMAGE_DIR, filename))

TEST_IMAGE_DIR = "/Users/lukaspicek/Downloads/images/FungiTastic-Mini/val/500p"

test_df["image_path"] = test_df.filename.apply(
    lambda filename: os.path.join(TEST_IMAGE_DIR, filename))
from sklearn import preprocessing

label_encoders = {}
columns_to_be_encoded = ["habitat", "substrate", "metaSubstrate"]

for column_name in columns_to_be_encoded:
    le = preprocessing.LabelEncoder()
    le.fit(train_df[column_name].unique())
    label_encoders = {column_name: le}

    train_df[column_name] = le.transform(train_df[column_name]).astype(np.int64)
    test_df[column_name] = le.transform(test_df[column_name]).astype(np.int64)
metadata = pd.concat([train_df, test_df])
metadata = metadata.drop_duplicates(subset="observationID")
len(metadata)

30473

# Calculating prios

In [5]:
TARGET_FEATURE = "category_id"

cls_counts = metadata.groupby(TARGET_FEATURE).size()
class_distribution = cls_counts / len(metadata)
sum(class_distribution)



1.0000000000000004

## Calculate Distributions of Selected Features

In [6]:
from utils.matadata_processing import get_target_to_feature_conditional_distributions

SELECTED_FEATURES = ["habitat", "month", "substrate", "metaSubstrate"]

# test_df = test_df[~test_df[SELECTED_FEATURES].isna().any(axis=1)]

metadata_distributions = {}
for feature in SELECTED_FEATURES:
    metadata_distributions[feature] = get_target_to_feature_conditional_distributions(
        metadata,
        feature,
        TARGET_FEATURE,
        add_to_missing=False
    )

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.17 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


# Predictions

## 1. Loading model from HuggingFace hub ⏳

In [7]:
N_CLASSES = len(metadata[TARGET_FEATURE].unique())
IMAGE_SIZE = [224, 224]

MODEL_NAME = "BVRA/vit_base_patch16_224.in1k_ft_fungitastic_224" #

USE_CALIBRATION = True
USE_OBSERVATION_PREDS = True

model = timm.create_model(f"hf-hub:{MODEL_NAME}", pretrained=True)
model = model.eval()

# model_mean = list(model.default_cfg['mean'])
# model_std = list(model.default_cfg['std'])
# print(model_mean, model_std)
model_mean =  [0.5, 0.5, 0.5]
model_std = [0.5, 0.5, 0.5]

print(model_mean, model_std)

model.to(device)
model.eval()
print(f"Done. {device}")

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (BVRA/vit_base_patch16_224.in1k_ft_fungitastic_224)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/timm/models/_hub.py:190: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded f

[0.5, 0.5, 0.5] [0.5, 0.5, 0.5]
Done. mps


In [8]:
from fgvc.special.calibration import ModelWithTemperature, get_temperature

if USE_CALIBRATION:
    model = ModelWithTemperature(model)
    model.to(device)

## 2. Prepare Dataloader

In [9]:
from utils.DanishFungiDataset import DanishFungiDataset, get_transforms

test_dataset = DanishFungiDataset(
    test_df,
    image_path_feature='image_path',
    target_feature=TARGET_FEATURE,
    extra_features=[*SELECTED_FEATURES, "observationID"], 
    transform=get_transforms(model_mean, model_std, IMAGE_SIZE)
)

In [10]:
test_df

,eventDate,year,month,day,habitat,countryCode,scientificName,kingdom,phylum,class,...,region,district,filename,category_id,metaSubstrate,poisonous,elevation,landcover,biogeographicalRegion,image_path
0,2022-01-01,2022.0,1.0,1.0,12,DK,Mycena chlorantha (Fr.) P.Kumm.,Fungi,Basidiomycota,Agaricomycetes,...,Sjælland,Solrød,0-3424503353.JPG,72,6,0,98.0,9.0,continental,/Users/lukaspicek/Downloads/images/FungiTastic...
1,2022-01-01,2022.0,1.0,1.0,10,DK,Clitocybe vibecina (Fr.) Quél.,Fungi,Basidiomycota,Agaricomycetes,...,Midtjylland,Silkeborg,0-3424490362.JPG,60,3,0,122.0,1.0,atlantic,/Users/lukaspicek/Downloads/images/FungiTastic...
2,2022-01-01,2022.0,1.0,1.0,10,DK,Clitocybe vibecina (Fr.) Quél.,Fungi,Basidiomycota,Agaricomycetes,...,Midtjylland,Silkeborg,1-3424490362.JPG,60,3,0,122.0,1.0,atlantic,/Users/lukaspicek/Downloads/images/FungiTastic...
3,2022-01-02,2022.0,1.0,2.0,25,DK,Mycena vitilis (Fr.) Quél.,Fungi,Basidiomycota,Agaricomycetes,...,Sjælland,Køge,0-3429085305.JPG,120,8,0,104.0,5.0,continental,/Users/lukaspicek/Downloads/images/FungiTastic...
4,2022-01-02,2022.0,1.0,2.0,25,DK,Mycena vitilis (Fr.) Quél.,Fungi,Basidiomycota,Agaricomycetes,...,Sjælland,Køge,1-3429085305.JPG,120,8,0,104.0,5.0,continental,/Users/lukaspicek/Downloads/images/FungiTastic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9407,2022-12-29,2022.0,12.0,29.0,5,DK,Mycena juniperina Aronsen,Fungi,Basidiomycota,Agaricomycetes,...,Nordjylland,Mariagerfjord,0-4100094061.JPG,87,8,0,0.0,17.0,continental,/Users/lukaspicek/Downloads/images/FungiTastic...
9408,2022-12-29,2022.0,12.0,29.0,5,DK,Mycena juniperina Aronsen,Fungi,Basidiomycota,Agaricomycetes,...,Nordjylland,Mariagerfjord,1-4100094061.JPG,87,8,0,0.0,17.0,continental,/Users/lukaspicek/Downloads/images/FungiTastic...
9409,2022-12-29,2022.0,12.0,29.0,5,DK,Mycena juniperina Aronsen,Fungi,Basidiomycota,Agaricomycetes,...,Nordjylland,Mariagerfjord,2-4100094061.JPG,87,8,0,0.0,17.0,continental,/Users/lukaspicek/Downloads/images/FungiTastic...
9410,2022-10-04,2022.0,10.0,4.0,4,DK,Amanita muscaria (L.) Lam.,Fungi,Basidiomycota,Agaricomycetes,...,Sjælland,Køge,0-4100104275.JPG,33,6,1,74.0,12.0,continental,/Users/lukaspicek/Downloads/images/FungiTastic...


In [11]:
batch_size = 64

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

## Inference with pre-trained model

In [12]:
from utils.matadata_processing import predict_with_features

preds, preds_raw, GT_lbls, seen_features = predict_with_features(model, test_loader, device)

vanilla_f1 = f1_score(test_df[TARGET_FEATURE], preds, average='macro')
vanilla_accuracy = accuracy_score(test_df[TARGET_FEATURE], preds)
print('Vanilla:', vanilla_f1, vanilla_accuracy)
vanilla_recall_3 = top_k_accuracy_score(test_df[TARGET_FEATURE], preds_raw, k=3, labels=labels)
vanilla_recall_5 = top_k_accuracy_score(test_df[TARGET_FEATURE], preds_raw, k=5, labels=labels)
vanilla_recall_10 = top_k_accuracy_score(test_df[TARGET_FEATURE], preds_raw, k=10, labels=labels)

print('Vanilla:', vanilla_f1, vanilla_accuracy, vanilla_recall_3, vanilla_recall_5, vanilla_recall_10)

  0%|          | 0/148 [00:00<?, ?it/s]INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.17 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.17 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.17 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.17 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable 

Vanilla: 0.00030176489348527475 0.0009562260943476413


ValueError: Number of given labels (215) not equal to the number of classes in 'y_score' (2829).

### Average image predictions to get an observation prediction

In [18]:
seen_observation_ids = np.array(seen_features["observationID"])
unique_observation_ids = np.unique(seen_observation_ids)

preds_raw_np = np.array(preds_raw)

obs_preds_raw = np.zeros((len(test_df), N_CLASSES))
obs_preds = np.zeros((len(test_df)))

for unique_observation_id in unique_observation_ids:
    same_observation_indexes = np.where(seen_observation_ids == unique_observation_id)
    
    observation_predictions = preds_raw_np[same_observation_indexes]
    _obs_preds = np.average(observation_predictions, axis=0)
    obs_preds_raw[same_observation_indexes] = _obs_preds
    obs_preds[same_observation_indexes] = _obs_preds.argmax()
    
obs_f1 = f1_score(test_df[TARGET_FEATURE], obs_preds, average='macro')
obs_accuracy = accuracy_score(test_df[TARGET_FEATURE], obs_preds)
obs_recall_3 = top_k_accuracy_score(test_df[TARGET_FEATURE], obs_preds_raw, k=3, labels=labels)

# print('ObservationID:', obs_f1, obs_accuracy, obs_recall_3)
print('ObservationID:', obs_f1, obs_accuracy)

if USE_OBSERVATION_PREDS:
    vanilla_f1 = obs_f1
    vanilla_accuracy = obs_accuracy
    vanilla_recall_3 = obs_recall_3
    preds_raw = obs_preds_raw


ObservationID: 0.560897894238223 0.8102422439439014


## Weighting by each Selected Feature

In [19]:
from utils.matadata_processing  import weight_predictions_by_feature_distribution

def post_process_selected_features(metadata_distributions, class_distribution, raw_predictions, ground_truth_labels):
    feature_prior_ratios = {}
    metrics_by_features = {}
    for feature in SELECTED_FEATURES:
        metadata_distribution = metadata_distributions[feature]
        seen_feature_values = seen_features[feature]

        weighted_predictions, weighted_predictions_raw, feature_prior_ratio = weight_predictions_by_feature_distribution(
            target_to_feature_conditional_distributions=metadata_distribution,
            target_distribution=class_distribution,
            ground_truth_labels=ground_truth_labels,
            raw_predictions=raw_predictions,
            ground_truth_feature_categories=seen_feature_values
        )
        feature_prior_ratios[feature] = feature_prior_ratio

        f1 = f1_score(test_df[TARGET_FEATURE], weighted_predictions, average='macro')
        accuracy = accuracy_score(test_df[TARGET_FEATURE], weighted_predictions)
        recall_3 = top_k_accuracy_score(test_df[TARGET_FEATURE], weighted_predictions_raw, k=3, labels=labels)
        metrics_by_features[feature] = {
            "f1": f1,
            "accuracy": accuracy,
            "recall_3": recall_3
        }
        print(f'{feature}:', f1, accuracy, recall_3)

        print(f'{feature} dif:', np.around(f1-vanilla_f1, 3), np.around((accuracy-vanilla_accuracy) * 100, 2), np.around((recall_3-vanilla_recall_3)*100))
        

    
    return feature_prior_ratios, metrics_by_features
        
feature_prior_ratios, metrics_by_features = post_process_selected_features(
    metadata_distributions=metadata_distributions,
    class_distribution=class_distribution,
    raw_predictions=preds_raw,
    ground_truth_labels=GT_lbls
)

100%|██████████| 9412/9412 [00:03<00:00, 2710.14it/s]


habitat: 0.474801004047195 0.7172758181045474 0.8764343391415215
habitat dif: 0.04 2.27 2.0


100%|██████████| 9412/9412 [00:03<00:00, 2657.99it/s]


month: 0.4458260616694827 0.703251168720782 0.8590097747556311
month dif: 0.011 0.87 1.0


100%|██████████| 9412/9412 [00:03<00:00, 2703.67it/s]


substrate: 0.4585104761407215 0.7060135996600085 0.8623034424139396
substrate dif: 0.023 1.15 1.0


100%|██████████| 9412/9412 [00:03<00:00, 2694.40it/s]


metaSubstrate: 0.45035787002378763 0.7033574160645983 0.8596472588185295
metaSubstrate dif: 0.015 0.88 1.0


## Weighting by Combinations of Selected Features

In [24]:
from itertools import combinations
from utils.matadata_processing import weight_predictions_combined_feature_priors


def post_process_prior_combinations(raw_predictions, feature_prior_ratios):
    metrics_by_combination = {}
    all_combinations_selected_features = []
    for num_features in range(2, len(SELECTED_FEATURES) + 1):
        all_combinations_selected_features.extend(combinations(SELECTED_FEATURES, num_features))
    
    for combination in all_combinations_selected_features:

        selected_feature_prior_ratios = [feature_prior_ratios[feature] for feature in combination]

        merged_predictions, merged_predictions_raw = weight_predictions_combined_feature_priors(
            raw_predictions=raw_predictions,
            feature_prior_ratios=selected_feature_prior_ratios
        )

        f1 = f1_score(test_df[TARGET_FEATURE], merged_predictions, average='macro')
        accuracy = accuracy_score(test_df[TARGET_FEATURE], merged_predictions)
        recall_3 = top_k_accuracy_score(test_df[TARGET_FEATURE], merged_predictions_raw, k=3, labels=labels)
        
        combination_name = " + ".join(combination)
        
        metrics_by_combination[combination_name] = {
            "f1": f1,
            "accuracy": accuracy,
            "recall_3": recall_3
        }
        print(combination_name)
        print("F1, Acc, Recall3: ", f1, accuracy, recall_3)

        print("Diff: ", np.around(f1-vanilla_f1, 3), np.around((accuracy-vanilla_accuracy) * 100, 2), np.around((recall_3-vanilla_recall_3)*100, 2))
    
    return metrics_by_combination
        
metrics_by_combination = post_process_prior_combinations(
    raw_predictions=preds_raw,
    feature_prior_ratios=feature_prior_ratios
)

habitat + month
F1, Acc, Recall3:  0.49515107947224063 0.7257756056098598 0.8812154696132597
Diff:  0.06 3.12 2.73
habitat + substrate
F1, Acc, Recall3:  0.4937887631435116 0.7243943901402465 0.8825966850828729
Diff:  0.059 2.99 2.87
habitat + metaSubstrate
F1, Acc, Recall3:  0.48604818900337304 0.7214194645133871 0.87972800679983
Diff:  0.051 2.69 2.58
month + substrate
F1, Acc, Recall3:  0.4751674888590399 0.7138759031024224 0.8686782830429239
Diff:  0.04 1.93 1.48
month + metaSubstrate
F1, Acc, Recall3:  0.46748332662556485 0.7102634934126647 0.8651721206969826
Diff:  0.032 1.57 1.13
substrate + metaSubstrate
F1, Acc, Recall3:  0.4620401821328909 0.7065448363790905 0.8615597110072248
Diff:  0.027 1.2 0.76
habitat + month + substrate
F1, Acc, Recall3:  0.5098980874467962 0.7307692307692307 0.888121546961326
Diff:  0.075 3.62 3.42
habitat + month + metaSubstrate
F1, Acc, Recall3:  0.5032994966766682 0.7273693157671058 0.8850403739906503
Diff:  0.068 3.28 3.11
habitat + substrate + met

In [27]:
results = {
    "Vanilla":       {'f1': vanilla_f1, 'accuracy': vanilla_accuracy, 'recall_3': vanilla_recall_3},
    "ObservationID": {'f1': obs_f1, 'accuracy': obs_accuracy, 'recall_3': obs_recall_3}
}

results.update(metrics_by_features)
results.update(metrics_by_combination)

results_df = pd.DataFrame(results).transpose()
results_df = results_df[['accuracy', 'f1', 'recall_3']]
results_df.head(50)

,accuracy,f1,recall_3
Vanilla,0.694539,0.435252,0.853910
ObservationID,0.810242,0.560898,0.932214
habitat,0.717276,0.474801,0.876434
month,0.703251,0.445826,0.859010
substrate,0.706014,0.458510,0.862303
metaSubstrate,0.703357,0.450358,0.859647
habitat + month,0.725776,0.495151,0.881215
habitat + substrate,0.724394,0.493789,0.882597
habitat + metaSubstrate,0.721419,0.486048,0.879728
month + substrate,0.713876,0.475167,0.868678


In [30]:
(results_df - results_df.iloc[0, :]) * 100

,accuracy,f1,recall_3
Vanilla,0.000000,0.000000,0.000000
ObservationID,11.570336,12.564606,7.830429
habitat,2.273693,3.954917,2.252444
month,0.871228,1.057423,0.509987
substrate,1.147471,2.325865,0.839354
metaSubstrate,0.881853,1.510604,0.573736
habitat + month,3.123672,5.989925,2.730557
habitat + substrate,2.985550,5.853693,2.868678
habitat + metaSubstrate,2.688058,5.079636,2.581810
month + substrate,1.933702,3.991566,1.476838


In [18]:
results_df.iloc[0, :]

accuracy    0.810242
f1          0.560898
Name: Vanilla, dtype: float64

In [49]:
from utils.matadata_processing import late_metadata_fusion

results = late_metadata_fusion(
    metadata,
    model,
    test_loader,
    labels,
    device,
    TARGET_FEATURE,
    SELECTED_FEATURES
)

  0%|          | 0/1401 [00:00<?, ?it/s]INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable

Unexpected exception formatting exception. Falling back to standard exception


  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/multiprocessing/util.py", line 360, in _exit_function
    _run_finalizers()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/multiprocessing/queues.py", line 201, in _finalize_join
    thread.join()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/threading.py", line 1053, in join
    self._wait_for_tstate_lock()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/threading.py", line 1069, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt
Tra

NameError: name 'merged_predictions_raw' is not defined